In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import numpy
import matplotlib.pyplot as plt
from scipy.stats import skew
from scipy.stats.stats import pearsonr
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from collections import OrderedDict
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
import xgboost as xgb

# %config InlineBackend.figure_format = 'retina' #set 'png' here when working on notebook
# %matplotlib inline

In [2]:
train = pd.read_csv('../normalized_train.csv',
        dtype={'gimnasio': int,
                'usosmultiples': int,
                'escuelascercanas': int,
                'piscina': int,
                'centroscomercialescercanos': int,

            },
        parse_dates=['fecha'])
test = pd.read_csv('../normalized_test.csv',
        dtype={'gimnasio': int,
                'usosmultiples': int,
                'escuelascercanas': int,
                'piscina': int,
                'centroscomercialescercanos': int,

            },
        parse_dates=['fecha'])
train_raw = pd.read_csv('train.csv',
        dtype={'gimnasio': int,
                'usosmultiples': int,
                'escuelascercanas': int,
                'piscina': int,
                'centroscomercialescercanos': int,
                'tipodepropiedad': 'category',
                'provincia': 'category',
                'ciudad': 'category',

            },
        parse_dates=['fecha'])
test = test.set_index('id').dropna(subset=['titulo'])
train = train.set_index('id').dropna(subset=['titulo'])
df_all = train.append(test)
train.shape

(234521, 43)

In [3]:
tipodepopiedades = [item.lower() for item in train_raw.tipodepropiedad.cat.categories]
tipodepopiedades

['apartamento',
 'bodega comercial',
 'casa',
 'casa en condominio',
 'casa uso de suelo',
 'departamento compartido',
 'duplex',
 'edificio',
 'huerta',
 'inmuebles productivos urbanos',
 'local comercial',
 'local en centro comercial',
 'lote',
 'nave industrial',
 'oficina comercial',
 'otros',
 'quinta vacacional',
 'rancho',
 'terreno',
 'terreno comercial',
 'terreno industrial',
 'villa',
 'hospedaje',
 'garage']

In [4]:
import nltk
import re
from unicodedata import normalize
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
#nltk.download('stopwords')
stopwords = stopwords.words('spanish')


from collections import Counter
counter = Counter()
tokenizer = RegexpTokenizer(r'\w+')
a,b = 'áéíóúü','aeiouu'
trans = str.maketrans(a,b)

def countplease(x):
    
#     print(x)
    if(x!=x): return;
    
    x = tokenizer.tokenize(x)

    for word in list(x):  # iterating on a copy since removing will mess things up
        if word in stopwords or word.isnumeric() or word in tipodepopiedades:
            x.remove(word)

    x = [item.translate(trans) for item in x]
    counter.update(x)
    

df_all['titulo'].apply(lambda x: countplease(x))
counter.most_common(20)


[('venta', 132690),
 ('departamento', 47720),
 ('san', 20543),
 ('excelente', 15632),
 ('lomas', 13890),
 ('col', 13448),
 ('residencial', 12570),
 ('hermosa', 11738),
 ('condominio', 10751),
 ('valle', 9904),
 ('recamaras', 8851),
 ('oportunidad', 8725),
 ('santa', 8598),
 ('fracc', 8115),
 ('fraccionamiento', 7559),
 ('nueva', 7399),
 ('casas', 7364),
 ('remate', 7127),
 ('bonita', 6797),
 ('cerca', 6441)]

In [5]:
soloprecio = train[['logprecio', 'metros']]
solopreciotrain = soloprecio;
solotest = test[['metros']]
def buildDataframe(appearences, counter):
    newDict = dict()
    # Iterate over all the items in dictionary and filter items which has even keys
    for (key, value) in dict(counter).items():
        # Check if key is even then add pair to new dictionary
    #     print(value)
        if int(value) > appearences:
            newDict[key] = value

    # print('Filtered Dictionary : ')
    print(len(newDict.keys()))
    columns = list(newDict.keys())


    for column in columns: 
        soloprecio[column] = train['titulo'].str.contains(column)
        solotest[column] = test['titulo'].str.contains(column)
    #     columncontent = []
    return int(len(newDict.keys()))


In [6]:
clf = MLPRegressor(hidden_layer_sizes=(10), activation='tanh', solver='adam')

In [7]:
from sklearn.model_selection import cross_val_score
columns = []
def rmse_cv(model, appearence):
    columns.append(buildDataframe(appearence, counter))

    
    every_column_except_y= [col for col in solopreciotrain.columns if col not in ['id', 'logprecio']]
    X_train, X_test, y_train, y_test = train_test_split(solopreciotrain[every_column_except_y], solopreciotrain.logprecio, test_size=0.2, random_state=123)
    print(solopreciotrain.logprecio.isnull().sum())
    rmse= np.sqrt(-cross_val_score(model, X_train, y_train, scoring="neg_mean_squared_error", cv = 2))
    return(rmse)

def xgBoost_rmse(model, appearence):
    columns.append(buildDataframe(appearence, counter))
    
    every_column_except_y= [col for col in solopreciotrain.columns if col not in ['id', 'logprecio']]
    X_train, X_test, y_train, y_test = train_test_split(solopreciotrain[every_column_except_y], solopreciotrain.logprecio, test_size=0.2, random_state=123)
    print(solopreciotrain.logprecio.isnull().sum())
    model.fit(X_train, y_train)
    
    print(np.sqrt(mean_squared_error(y_test, model.predict(X_test))))
    print(OrderedDict(sorted(model.get_booster().get_fscore().items(), key=lambda t: t[1], reverse=True)))
    return (OrderedDict(sorted(model.get_booster().get_fscore().items(), key=lambda t: t[1], reverse=True)))
#     return(rmse)

appearences = list(range(2000, 0, -200))
# cv_neuron = [rmse_cv(clf, appearence).mean() for appearence in appearences]

In [8]:
model = xgb.XGBRegressor(learning_rate =0.1, n_estimators=700, max_depth=5, gamma=0, subsample=0.8, colsample_bytree=0.8, nthread=6, scale_pos_weight=1, seed=27)
important_words = xgBoost_rmse(model, 800)

266


/home/rama/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/rama/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


0


/home/rama/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[18:14:40] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1572314959925/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
0.5022790505518927
OrderedDict([('metros', 4888), ('departamento', 822), ('venta', 481), ('c', 299), ('d', 294), ('col', 234), ('san', 230), ('valle', 188), ('i', 179), ('l', 175), ('depto', 167), ('av', 163), ('local', 160), ('condominio', 157), ('fracc', 157), ('m2', 156), ('lomas', 152), ('remate', 137), ('n', 137), ('residencial', 134), ('pre', 133), ('fe', 133), ('excelente', 125), ('comercial', 121), ('min', 111), ('centro', 111), ('hermosa', 109), ('santa', 105), ('rio', 104), ('departamentos', 100), ('rec', 100), ('loma', 99), ('oficina', 97), ('gran', 93), ('torre', 93), ('ciudad', 93), ('nuevo', 91), ('residencia', 90), ('norte', 88), ('juriquilla', 88), ('vista', 85), ('casas', 82), ('sur', 78), ('oportunidad', 76), ('sol', 75), ('zona', 74), ('mar', 72), ('cerca', 71), ('pesos', 70), ('calle

In [9]:
# clffinal = MLPRegressor(activation='tanh', solver='adam')
# buildDataframe(15000)
# every_column_except_y= [col for col in solopreciotrain.columns if col not in ['id', 'logprecio']]
# X_train, X_test, y_train, y_test = train_test_split(solopreciotrain[every_column_except_y], solopreciotrain.logprecio, test_size=0.2, random_state=123)

In [10]:
# cv_neuron = pd.Series(cv_neuron, index = columns)
# cv_neuron.plot(title = "Estimación solo con palabras del titulo")
# plt.xlabel("Columnas")
# plt.ylabel("rmse")
# cv_neuron

In [11]:
# X_train, X_test, y_train, y_test = train_test_split(solopreciotrain[every_column_except_y], solopreciotrain.logprecio, test_size=0.2, random_state=123)
# np.sqrt(mean_squared_error(y_test, clf.predict(y_train)))

In [12]:
clffinal = MLPRegressor(activation='tanh', solver='adam')
#buildDataframe(15000)
every_column_except_y= [col for col in solopreciotrain.columns if col not in ['id', 'logprecio']]
X_train, X_test, y_train, y_test = train_test_split(solopreciotrain[every_column_except_y], solopreciotrain.logprecio, test_size=0.2, random_state=123)

In [13]:
# clffinal.fit(X_train, y_train)

In [14]:
# np.sqrt(mean_squared_error(y_test, clffinal.predict(X_test)))

In [15]:
buildDataframe(50, Counter(important_words).most_common(100))

76


76

In [16]:
from sklearn.feature_extraction import FeatureHasher

cantidad_features = 20

h = FeatureHasher(n_features=cantidad_features, input_type='string')
every_column_except_y= [col for col in solopreciotrain.columns if col not in ['id', 'logprecio', 'metros']]

train_hashtrick = solopreciotrain.copy()
test_hashtrick = solotest.copy()

# Recorro las columnas y asigno la palabra si es True, sino nan.
for el in every_column_except_y:
    train_hashtrick.loc[train_hashtrick[el] == True, el] = el
    train_hashtrick.loc[train_hashtrick[el] == False, el] = np.nan
    test_hashtrick.loc[test_hashtrick[el] == True, el] = el
    test_hashtrick.loc[test_hashtrick[el] == False, el] = np.nan

# Armo la matriz de arrays para poder usar the hashing trick
train_hashtrick = train_hashtrick[every_column_except_y].apply(lambda x: list(filter(lambda y : y == y, x)), axis=1)
test_hashtrick = test_hashtrick[every_column_except_y].apply(lambda x: list(filter(lambda y : y == y, x)), axis=1)

# Termino de armar los arrays.
names = [f'fh{el + 1}' for el in range(cantidad_features)]
f = h.transform(train_hashtrick.values)
train_hashtrick = pd.DataFrame(f.toarray(), columns=names)
train_hashtrick['id'] = train.index
train_hashtrick = train_hashtrick.set_index('id')
f = h.transform(test_hashtrick.values)
test_hashtrick = pd.DataFrame(f.toarray(), columns=names)
test_hashtrick['id'] = test.index
test_hashtrick = test_hashtrick.set_index('id')
test_hashtrick

,fh1,fh2,fh3,fh4,fh5,fh6,fh7,fh8,fh9,fh10,fh11,fh12,fh13,fh14,fh15,fh16,fh17,fh18,fh19,fh20
id,,,,,,,,,,,,,,,,,,,,
4941,0.0,-2.0,-2.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,-1.0,0.0,1.0
51775,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
115253,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-1.0,0.0,1.0,-1.0,0.0,1.0
299321,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-1.0,0.0,-1.0,0.0,1.0
173570,0.0,-1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,2.0,0.0,1.0,-1.0,0.0,-1.0,-1.0,1.0
30862,-2.0,-2.0,0.0,0.0,-2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,-1.0,-1.0,0.0,1.0
244471,0.0,-2.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-1.0,0.0,0.0,-1.0,0.0,0.0
127794,-2.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,-1.0,-2.0,-1.0,-1.0,1.0
71558,0.0,0.0,1.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
test_hashtrick.shape

(58622, 20)

In [18]:
finaltest = pd.read_csv('../normalized_test.csv')
finaltest = finaltest.set_index('id')
finaltest = finaltest.join(test_hashtrick, how='left')

finaltest.loc[:, names] = finaltest[names].fillna(0)
finaltest.to_csv('../normalized2_test.csv')

In [19]:
finaltrain = pd.read_csv('../normalized_train.csv')
finaltrain = finaltrain.set_index('id')
finaltrain = finaltrain.join(train_hashtrick)

finaltrain.loc[:, names] = finaltrain[names].fillna(0)
finaltrain.to_csv('../normalized2_train.csv')

In [20]:
finaltrain.precio

id
254099     2273000.0
53461      3600000.0
247984     1200000.0
209067      650000.0
185997     1150000.0
126147     1100000.0
139233     1150000.0
5013       4200000.0
44962       310000.0
134537     6200000.0
51180       488000.0
103293     7200000.0
47890      1500000.0
130216     3000000.0
181436     5300000.0
201923     3650000.0
283945     2450000.0
73348      4750000.0
40421       794078.0
87655      1850000.0
90071       490000.0
224513     2300000.0
129818     1340000.0
146699     3500000.0
235958     3900000.0
97918       850000.0
70294       310000.0
177031    10800000.0
144635     2600000.0
146158    10707900.0
             ...    
123855     3350000.0
2747       3750000.0
130523     2400000.0
149503     1260000.0
156730     3300000.0
258795     1275000.0
184779     2020000.0
214176      490000.0
235796     1880000.0
103355     3300000.0
267455     3386400.0
199041      550000.0
252709      357000.0
194027      684338.0
262913      505000.0
64820      2700000.0
41090     

In [21]:
finaltrain.shape

(239907, 63)